In [1]:
import pandas as pd
import numpy as np
import feature_engine 
import pickle
import matplotlib.pyplot as plt

In [ ]:
ENCODED_DATA_PATH = '../data/processed/1_encoded_df.pkl'
TARGET_COL = 'DevType'
ENGINEERED_DF_PATH = '../data/processed/2_prepocessed_df.pkl'

In [ ]:
df = pd.read_pickle(ENCODED_DATA_PATH)
